In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("C:/Users/Report/Desktop/New folder/car_sales_data.csv")

In [3]:
df.head()

,Manufacturer,Model,Engine size,Fuel type,Year of manufacture,Mileage,Price
0,Ford,Fiesta,1.0,Petrol,2002,127300,3074
1,Porsche,718 Cayman,4.0,Petrol,2016,57850,49704
2,Ford,Mondeo,1.6,Diesel,2014,39190,24072
3,Toyota,RAV4,1.8,Hybrid,1988,210814,1705
4,VW,Polo,1.0,Petrol,2006,127869,4101


In [4]:
df.tail()

,Manufacturer,Model,Engine size,Fuel type,Year of manufacture,Mileage,Price
49995,BMW,M5,5.0,Petrol,2018,28664,113006
49996,Toyota,Prius,1.8,Hybrid,2003,105120,9430
49997,Ford,Mondeo,1.6,Diesel,2022,4030,49852
49998,Ford,Focus,1.0,Diesel,2016,26468,23630
49999,VW,Golf,1.4,Diesel,2012,109300,10400


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Manufacturer         50000 non-null  object 
 1   Model                50000 non-null  object 
 2   Engine size          50000 non-null  float64
 3   Fuel type            50000 non-null  object 
 4   Year of manufacture  50000 non-null  int64  
 5   Mileage              50000 non-null  int64  
 6   Price                50000 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 2.7+ MB


In [6]:
df.describe()

,Engine size,Year of manufacture,Mileage,Price
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,1.773058,2004.209440,112497.320700,13828.903160
std,0.734108,9.645965,71632.515602,16416.681336
min,1.000000,1984.000000,630.000000,76.000000
25%,1.400000,1996.000000,54352.250000,3060.750000
50%,1.600000,2004.000000,100987.500000,7971.500000
75%,2.000000,2012.000000,158601.000000,19026.500000
max,5.000000,2022.000000,453537.000000,168081.000000


In [7]:
print(df.shape)

(50000, 7)


In [8]:
df.isnull().sum()

Manufacturer           0
Model                  0
Engine size            0
Fuel type              0
Year of manufacture    0
Mileage                0
Price                  0
dtype: int64

In [9]:
df.duplicated().sum()

12

In [10]:
df[df.duplicated()]

,Manufacturer,Model,Engine size,Fuel type,Year of manufacture,Mileage,Price
5426,VW,Polo,1.2,Petrol,2003,10000,8024
9862,Ford,Mondeo,1.4,Diesel,1987,224569,883
14745,BMW,Z4,2.4,Petrol,1999,12000,13410
19020,Toyota,Yaris,1.0,Petrol,1996,13500,5087
19337,VW,Polo,1.0,Petrol,2000,11500,5950
23927,VW,Polo,1.2,Petrol,2021,1000,27901
25368,VW,Golf,1.2,Diesel,2011,6000,17401
28576,VW,Polo,1.2,Petrol,2003,10000,8024
34246,VW,Passat,2.0,Diesel,2003,10000,16087
35647,Ford,Focus,1.6,Petrol,2019,2000,39636


In [11]:
df = df.drop_duplicates()

In [12]:
print("New dataset shape:", df.shape)

New dataset shape: (49988, 7)


In [13]:
import datetime
df["CarAge"] = datetime.datetime.now().year - df["Year of manufacture"]

In [14]:
df = df[["Manufacturer", "Model", "Engine size", "Fuel type", "Year of manufacture", "CarAge", "Mileage", "Price"]]

## data overview

## Modeling Insight

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [16]:
df = df.dropna(subset=['Price'])
df['Engine size'] = pd.to_numeric(df['Engine size'], errors='coerce')
df = df.dropna()

In [17]:
# Features (X) and target (y)
X = df.drop(columns=["Price"])
y = df["Price"]

In [18]:
cat_cols = ['Manufacturer', 'Model', 'Fuel type']
num_cols = ['Engine size', 'Year of manufacture', 'Mileage']
pre = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols), ('num', 'passthrough', num_cols)])

In [34]:
# Train model
model1 = RandomForestRegressor(n_estimators= 10, max_depth=None, random_state=42, n_jobs=-1)
pipe=Pipeline(steps=[('prep', pre), ('model', model1)])

In [35]:
# Copy dataset
#df_model = df.copy()

# Encode categorical variables
#for col in ["Manufacturer", "Model", "Fuel type"]:
#    df_model[col] = LabelEncoder().fit_transform(df_model[col])



# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)

,steps,"[('prep', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [36]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Predictions
pred = pipe.predict(X_test)

# Evaluation metrics
rmse = np.sqrt(mean_squared_error(y_test, pred))
r2 = r2_score(y_test, pred)

print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")


Root Mean Squared Error (RMSE): 711.88
R² Score: 1.00


In [37]:
import joblib

# Save the model
joblib.dump(pipe, 'car_model.pkl')
print("\nModel saved as 'car_sales_model.pkl'")



Model saved as 'car_sales_model.pkl'


In [38]:
pipe = joblib.load('car_model.pkl')
print(type(pipe))
print(dir(pipe))

<class 'sklearn.pipeline.Pipeline'>
['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__sklearn_clone__', '__sklearn_is_fitted__', '__sklearn_tags__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_build_request_for_signature', '_can_fit_transform', '_can_inverse_transform', '_can_transform', '_check_method_params', '_doc_link_module', '_doc_link_template', '_doc_link_url_param_generator', '_estimator_type', '_final_estimator', '_fit', '_get_default_requests', '_get_doc_link', '_get_metadata_for_step', '_get_metadata_request', '_get_param_names', '_get_params', '_get_params_html', '_html_repr', '_iter', '_log_me

In [39]:
import os
print(os.path.exists('car_model.pkl'))

True


In [40]:
import pickle

In [41]:
with open("car1.pkl", 'wb') as file:
    pickle.dump(pipe, file)

In [42]:
with open("car1.pkl", 'rb') as file:
    loaded_model = pickle.load(file)
    

In [43]:
print("Model training and evaluate completed successfully!")

Model training and evaluate completed successfully!
